# Visualize Duckietown watchtower trajectory

### Importing and loading all the needed modules

In [ ]:
import contracts
import yaml
import numpy as np
import geometry as geo
import numpy as np
from duckietown_world.world_duckietown.tile_template import load_tile_types
from duckietown_world.geo.measurements_utils import iterate_by_class
from duckietown_world.world_duckietown.tile import get_lane_poses
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_svg, ipython_draw_html
import duckietown_world as dw
from copy import deepcopy
import geometry as g
import pandas as pd
import sys
import traceback

contracts.disable_all()
m = dw.load_map('robotarium2')


### Setting up necessary classes and functions

In [ ]:
class Person(dw.PlacedObject):
    
    def __init__(self, radius, *args, **kwargs):
        self.radius = radius
        dw.PlacedObject.__init__(self, *args, **kwargs)

    def draw_svg(self, drawing, g):
        # drawing is done using the library svgwrite
        c = drawing.circle(center=(0, 0), r=self.radius, fill='pink')
        g.add(c)
        # draws x,y axes
        dw.draw_axes(drawing, g)

    def extent_points(self):
        # set of points describing the boundary 
        L = self.radius
        return [(-L, -L), (+L, +L)]

def relative_pose(q0, q1):
    return g.SE2.multiply(g.SE2.inverse(q0), q1)

def interpolate(q0, q1, alpha):
    q1_from_q0 = relative_pose(q0, q1)
    vel = g.SE2.algebra_from_group(q1_from_q0)
    rel = g.SE2.group_from_algebra(vel * alpha)
    q = g.SE2.multiply(q0, rel)
    return q

### Importing trajectory files and extract relevant data

In [ ]:
root = dw.PlacedObject()

realTimestamps = []
seqs2 = []
final_trajectory = []
startTime = 0

# Folder to trajectory for visualization
folderNamesSingle = 'autobot04_r2'

for fileName in range(0,500):
    trajectoryFile = 'trajectoryFiles/' + str(folderNamesSingle) + '/' + str(folderNamesSingle) + '_' + str(fileName) + '.yaml'

    try:
        with open(trajectoryFile, 'r') as stream:
            try:
                data = yaml.safe_load(stream)

            except yaml.YAMLError as exc:
                print(exc)
    except:
        if fileName !=0:
            break

    timestart = data['begin_time_stamp']
    data_points = len(data['trajectory_data'])

    x = np.zeros((data_points,))
    y= np.zeros((data_points,))
    R = np.zeros((3,3, data_points))            
    phi = np.zeros((3, data_points))

    dx = 999.999*np.ones((data_points, ))
    dy = 999.999*np.ones((data_points, ))

    dr = 999.999*np.ones((data_points, ))
    dphi = 999.999*np.ones((data_points, ))


    for idx, [time, traj] in enumerate(data['trajectory_data'].items()):

        x[idx] = np.array(traj[0])

        y[idx] = np.array(traj[1])

        R[:,:,idx] = np.reshape(np.array(traj[3:]), (3,3))


        phi[:,idx] = np.array([np.arctan2(-R[1,2,idx],R[2,2,idx]), 
                               np.arctan2(R[0,2,idx],np.sqrt(R[0,0,idx]**2 + R[0,1,idx]**2)),
                               np.arctan2(-R[0,1,idx], R[0,0,idx])])

        realTimestamps.append(float(time) + float(timestart))
        z = phi[2,idx]
        points = np.array([x[idx], y[idx]])
        final_trajectory.append([points, z])
    
final_array = final_trajectory

for entry in range(0, len(final_array)):
    x =  (final_array[entry][0][0] )  # -2.2
    y = final_array[entry][0][1] # + 0.8
    alpha = final_array[entry][1]
    q5 = geo.SE2_from_translation_angle([x,y],alpha)
    seqs2.append(q5)



### Calculate distance and relative heading to each timestamp

In [ ]:
timestamps = range(len(seqs2)) # [0, 1, 2, ...]

# SE2Transform is the wrapper for SE2 used by Duckietown World 
transforms = [dw.SE2Transform.from_SE2(_) for _ in seqs2]
seq_me = dw.SampledSequence(timestamps, transforms)
counter = 0

for timestamp, pose_object in seq_me:
    print(counter)
    lanePoses = list(get_lane_poses(m, pose_object.as_SE2()))
    
    if len(lanePoses) == 0:
        print('')
        
    else:
        distance_from_center = lanePoses[0].lane_pose.distance_from_center
        rel_heading = lanePoses[0].lane_pose.relative_heading
        tile = list(lanePoses[0].tile.children.keys())[0]
        correctDir = lanePoses[0].lane_pose.correct_direction

    counter += 1
    print('')


### Draw trajectory

In [ ]:
# Load map
m = dw.load_map('robotarium2')

m.set_object("emded", Person(0.1), ground_truth=seq_me)
ipython_draw_html(m);
